In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import requests
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from matplotlib import pyplot


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SHASHWAT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("D:/Recommender System Datasets/lthing_data/reviews2.csv")
df=df[1:]
df = df.reset_index()

In [3]:
df

,index,Unnamed: 0,comment,nhelpful,unixtime,work,flags,user,stars,time
0,1,1,"Every evening, the brave queen of Persia, Shah...",0.0,1.332288e+09,73960.0,[],Elizabeth.Wong98,4.5,"Mar 21, 2012"
1,2,2,Its the 1983-84 school year and Edward Zanni o...,0.0,1.253405e+09,69413.0,[],rivkat,3.0,"Sep 20, 2009"
2,3,3,"I became privy to this book, when another auth...",0.0,1.299888e+09,9523995.0,[],suz.haugland,4.0,"Mar 12, 2011"
3,4,4,.,0.0,1.327277e+09,368228.0,[],amoskovacs,4.0,"Jan 23, 2012"
4,5,5,This is a tremendous story of a freed woman an...,0.0,1.329005e+09,11243828.0,[],CandyH,4.0,"Feb 12, 2012"
...,...,...,...,...,...,...,...,...,...,...
995,996,996,With an abundance of excellent books about the...,1.0,1.219882e+09,5474830.0,[],khuggard,2.0,"Aug 28, 2008"
996,997,997,http://www.imdb.com/title/tt71853/ http://www....,0.0,1.280794e+09,8015336.0,"['abuse', 'not_a_review']",WilliamHartPhD,NaN,"Aug 3, 2010"
997,998,998,"Carley DeCarlo, a financial advisor newspaper ...",0.0,1.154909e+09,31799.0,[],dutchmarbel,NaN,"Aug 7, 2006"
998,999,999,In the various discussions/writings about the ...,0.0,1.286410e+09,245554.0,[],cyderry,2.5,"Oct 7, 2010"


In [4]:
def _removeNonAscii(s):
    s=str(s)
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    text=str(text)
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df['cleaned'] = df['comment'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)

In [5]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())


In [6]:
model = Word2Vec(window = 10, sg = 1, hs = 0, negative = 10, alpha=0.03, min_alpha=0.0007, seed = 14)
model.build_vocab(corpus, progress_per=200)

model.train(corpus, total_examples = model.corpus_count, epochs=10, report_delay=1)


(561818, 818240)

In [7]:
watch = df[["work", "comment"]]
watch

,work,comment
0,73960.0,"Every evening, the brave queen of Persia, Shah..."
1,69413.0,Its the 1983-84 school year and Edward Zanni o...
2,9523995.0,"I became privy to this book, when another auth..."
3,368228.0,.
4,11243828.0,This is a tremendous story of a freed woman an...
...,...,...
995,5474830.0,With an abundance of excellent books about the...
996,8015336.0,http://www.imdb.com/title/tt71853/ http://www....
997,31799.0,"Carley DeCarlo, a financial advisor newspaper ..."
998,245554.0,In the various discussions/writings about the ...


In [8]:
watch_dict = watch.groupby('work')['comment'].apply(list).to_dict()


In [9]:
ms = model.wv.similar_by_vector('every', topn= 6)[0:]
ms

[('single', 0.7814009785652161),
 ('basically', 0.7355883717536926),
 ('sitting', 0.7347422242164612),
 ('drive', 0.7239097952842712),
 ('shut', 0.7146080136299133),
 ('ray', 0.7046696543693542)]

In [10]:
def vectors(x):
    global word_embeddings
    word_embeddings = []
    arr=[]
    for index, row in df.iterrows():
        line=row['cleaned']
        avgword2vec = None
        count = 0
        """
        for word in model.wv.key_to_index:
            if(word not in line.split()):
                arr.append(0)
            else:
                arr.append(model.wv.key_to_index[word])
        """
        
        for word in line.split():
            if word in model.wv:
                count += 1
                if avgword2vec is None:
                    avgword2vec = model.wv[word]
                else:
                    avgword2vec = avgword2vec + model.wv[word]
        
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        word_embeddings.append([row['work'],avgword2vec])

In [11]:
vectors(df)
len(word_embeddings)

1000

In [16]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 5, stop_words='english')
tfidf.fit(df['cleaned'])

tfidf_list = dict(zip(tfidf.get_feature_names_out(), list(tfidf.idf_)))
tfidf_feature = tfidf.get_feature_names_out() 

In [17]:
tfidf_vectors = []; 
line = 0;
c=0

for desc in corpus: 
    sent_vec = np.zeros(100) 
    weight_sum =0; 
    for word in desc: 
        if word in model.wv and word in tfidf_feature:
            vec = model.wv[word]
            tf_idf = tfidf_list[word] * (desc.count(word) / len(desc))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    #sent_vec=np.dot(sent_vec,float(df[c:c+1]["stars"])/5)
    tfidf_vectors.append(sent_vec)
    line += 1
    c=c+1

In [18]:
tfidf_vectors[:2]

[array([-0.0013789 ,  0.10112164,  0.08070884,  0.04530863,  0.2200386 ,
        -0.09461053,  0.14517801,  0.0711839 ,  0.05946843, -0.0176511 ,
        -0.33672504, -0.02596602, -0.01746343, -0.02399768, -0.59810513,
         0.02970209, -0.19732259,  0.24283683,  0.20141012,  0.05633372,
         0.09380304, -0.22820388,  0.181994  ,  0.12861067, -0.25533363,
        -0.26072453, -0.09883572,  0.11904398,  0.41778117, -0.02524871,
        -0.00272557,  0.09640317,  0.09347564,  0.0280498 ,  0.22212499,
         0.18388925,  0.0221814 , -0.12608345,  0.01197574, -0.09125508,
         0.06405934,  0.12379997,  0.1642075 ,  0.19254474,  0.05030376,
        -0.13009397, -0.33015665,  0.00808089,  0.21518292,  0.27417307,
        -0.25182562, -0.23951597, -0.00888445, -0.21379295, -0.08979685,
        -0.21266961, -0.03723563, -0.07474123,  0.11096589,  0.04146327,
         0.01229575,  0.12757401, -0.26001875, -0.02306685, -0.33471425,
        -0.38757889, -0.19720559,  0.09875544, -0.0

In [19]:
def recommendations(cleaned):
    cosine_similarities = cosine_similarity(tfidf_vectors,  tfidf_vectors)
    
    books = df[['work', 'cleaned', 'comment']]
    indices = pd.Series(df.index, index = df['cleaned']).drop_duplicates()
    idx = indices[cleaned]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:100]
    return sim_scores
    
        

In [20]:
recommendations(df['cleaned'][0])[:10]

[(429, 0.9672482510802841),
 (411, 0.9649649768318359),
 (232, 0.963883205371965),
 (229, 0.962734369097337),
 (524, 0.9622079387877907),
 (904, 0.9620909672250061),
 (892, 0.9612469582866209),
 (902, 0.9610452192603739),
 (981, 0.9610437281189856),
 (714, 0.9610345964074726)]

In [21]:
import requests
from bs4 import BeautifulSoup

def find_title(work):
    a=int(work)
    r = requests.get('https://www.librarything.com/work/'+str(a)+'/workdetails') 

    soup = BeautifulSoup(r.content, 'html.parser') 

    t=str(soup.title)
    t=t.replace("<title>Workdetails: ","")
    t=t.replace(" | LibraryThing</title>","")
    return t


In [22]:
def final_rec(user):
    books = df[['work', 'cleaned', 'comment','user']]
    dfn = df[df['user']==user]
    c=[]
    for index, row in dfn.iterrows():
        c.append(recommendations(row['cleaned']))

    k=[]
    for i in c:
        for j in i:
            k.append(j)

    k = sorted(k, key = lambda x: x[1], reverse = True)
    k = k[0:100]
    #[1,7,18,...]
    book_indices = [i[0] for i in k]
    book_indices=list(set(book_indices))
    
    
    recommend = list(books['work'].iloc[book_indices])
    r=[]
    hist=list(books[books['user']==user]['work'])
    for i in recommend:
        if(i not in hist):
            r.append(i)
        
            
    r=list(set(r))[0:5]
    for i in r:
        print(find_title(i))
        print(i)



In [23]:
final_rec('_Zoe_')

The Late, Lamented Molly Marx by Sally Koslow
6506505.0
The Case for Christ: A Journalist's Personal Investigation of the Evidence for Jesus by Lee Strobel
3106313.0
The Secret by Anna Enquist
1174539.0
The Undomestic Goddess by Sophie Kinsella
8205.0
Blindsight by Peter Watts
1333265.0
